In [1]:
from genepy.utils import helper as h

from taigapy import TaigaClient
from celligner import Celligner
tc = TaigaClient()
from bokeh.plotting import output_notebook
from depmapomics import tracker as track
#autoreload
%load_ext autoreload
%autoreload 2
output_notebook()

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:14: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "l2_norm" failed type inference due to: No implementation of function Function(<function norm at 0x7fce3037f158>) found for signature:
 
 >>> norm(x=array(float32, 2d, A), axis=Literal[int](1))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'norm_impl': File: numba/np/linalg.py: Line 2352.
    With argument(s): '(x=array(float32, 2d, A), axis=int64)':
   Rejected as the implementation raised a specific error:
     TypeError: norm_impl() got an unexpected keyword argument 'x'
  raise

/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:199: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "adjust_s_variance" failed type inference due to: NameError: name 'sq_dist_to_line' is not defined
  @jit(float32(float32[:, :], float32[:, :], float32[:], float32[:], float32), nogil=True)
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:199: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "adjust_s_variance" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", line 207:
def adjust_s_variance(data1, data2, curcell, curvect, sigma):
    <source elided>
    totalprob2 = 0.
    for samecell in data2:
    ^

  @jit(float32(float32[:, :], float32[:, :], float32[:], float32[:], float32), nogil=True)
/home/jeremie/miniconda3/lib/pytho

Loading BokehJS ...

In [2]:
# load from taiga public (figshare link)
# load internal expression,
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  # https://depmap.org/portal/download/
# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here:
TCGA_expression = tc.get(name='celligner-input-9827',
                         file='tumor_expression')

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [3]:
# subset gene names to ensembl ids only
CCLE_expression = CCLE_expression[CCLE_expression.columns[:-92]]
CCLE_expression.columns = list(map(lambda x: x.split(
    ' (')[1][:-1] if ' (' in x else x, CCLE_expression.columns))

common = set(CCLE_expression.columns).intersection(
    set(TCGA_expression.columns))
CCLE_expression = CCLE_expression[list(common)]
TCGA_expression = TCGA_expression[list(common)]

In [4]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY/edit#gid=746987867 
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually, found also here: 

No dataset version provided. Using version 1.


In [5]:
# transforming annotations
CCLE_annotation = CCLE_annotation.drop_duplicates('arxspan_id').set_index("arxspan_id")
CCLE_annotation = CCLE_annotation.loc[CCLE_expression.index, ["origin", 'subtype']].rename(columns={"origin": "tissue_type", "subtype": 'disease_type'})
CCLE_annotation["cell_type"] = "cancer cell line"

TCGA_annotation = TCGA_annotation.set_index("sampleID").loc[TCGA_expression.index,["lineage",
"subtype"]].rename({"lineage":"tissue_type", "subtype": 'disease_type'})
TCGA_annotation['cell_type'] = "tumor sample"

In [6]:
my_alligner = Celligner(snn_kwargs={"neighbor_num": 30, "min_shared_neighbor_proportion": 1/15,}, make_plots=True)

my_alligner.fit(TCGA_expression.iloc[:800], TCGA_annotation.iloc[:800])

fetching gene names from biomart cache
using only usefull genes
looking at 800 samples.
found 29593 common genes
creating a fit dataset..
reducing dimensionality...
clustering...


doing differential expression analysis on the clusters
running differential expression on 6 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
done


In [ ]:
my_alligner.transform(CCLE_expression.iloc[:400], CCLE_annotation.iloc[:400])

looking at 800 samples.
found 29593 common genes
creating a transform input..
reducing dimensionality...
clustering..


doing differential expression analysis on the clusters..
running differential expression on 5 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
there is 0.317 overlap between the fit and transform dataset in their most variable genes
doing cPCA..
transform
regressing out the cPCA components..
> /home/jeremie/celligner/celligner/__init__.py(360)transform()
-> varsubset = np.array([1 if i in self.differential_genes_names else 0 for i in self.transform_input.columns]).astype(bool)
(Pdb) n
> /home/jeremie/celligner/celligner/__init__.py(361)transform()
-> self.corrected = mnnpy.mnn_correct(transformed_fit.values.astype(float),
(Pdb) self.corrected = mnnpy.mnn_correct(transformed_fit.values.astype(float),                                       transformed_transform.values.astype(float),                                       var_index=list(transformed_fit.columns),                                       #var_subset=list(self.differentia

/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:88: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "find_mutual_nn" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py (94)

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", line 94:
def find_mutual_nn(data1, data2, k1, k2, n_jobs):
    <source elided>
    mutual_2 = []
    for index_2 in range(data2.shape[0]):
    ^

  @jit((float32[:, :], float32[:, :], int8, int8, int8))
/home/jeremie/miniconda3/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "find_mutual_nn" was compiled in object mode without forceobj=True.

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", line 94:
def find_mutual_nn(data1, data2, k1, k2, n_jobs):
    <source elided>
    mutual_2 = []
    for index_2 in range(data2.shap

  Computing correction vectors...


/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:102: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "compute_correction" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py (107)

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", line 107:
def compute_correction(data1, data2, mnn1, mnn2, data2_or_raw2, sigma):
    <source elided>
    vect_reduced = np.zeros((data2.shape[0], vect.shape[1]), dtype=np.float32)
    for index, ve in zip(mnn2, vect):
    ^

  @jit(float32[:, :](float32[:, :], float32[:, :], int32[:], int32[:], float32[:, :], float32))
/home/jeremie/miniconda3/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "compute_correction" was compiled in object mode without forceobj=True.

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", lin

*** IndexError: arrays used as indices must be of integer (or boolean) type
(Pdb) self.corrected = mnnpy.mnn_correct(transformed_fit,                                       transformed_transform,                                       var_index=list(transformed_fit.columns),                                       **self.mnn_kwargs)
Performing cosine normalization...
*** TypeError: No matching definition for argument type(s) pyobject, array(float32, 2d, C)
(Pdb) self.corrected = mnnpy.mnn_correct(transformed_fit.astype(float),                                       transformed_transform.astype(float),                                       var_index=list(transformed_fit.columns),                                       **self.mnn_kwargs)
Performing cosine normalization...
*** TypeError: No matching definition for argument type(s) pyobject, array(float32, 2d, C)
(Pdb) self.corrected = mnnpy.mnn_correct(transformed_fit.values.astype(float),                                       transformed_trans

In [ ]:
my_alligner.save('temp/demo/')

In [ ]:
my_alligner.plot()